<b>Autoinland is an insuarance company in west africa.
Their goal is to better improve customer experience and increase level of satisfaction when it comes to dealing with claims.
To met this goal, they want to anticipate future demands in terms of claim request volume which will also help them manage their resources.
To do this, we will help them by developing a predictive model that determines if a customer will submit a vehicle insuarance claim
in the next three months.
The solution will help streamline financial planning in autoinland.
it will alsohelp them better serve their customers by understanding which customers are likely to submit a claim.<b/>


## Import libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Import visualization libraries

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Reading the data

In [3]:
train=pd.read_csv('Train_insuarance.csv')
test=pd.read_csv('test_insuarance.csv')

## Combine train and test set

In [4]:
ntrain = train.shape[0] # to be used to split train and test set from the combined dataframe

df= pd.concat((train, test)).reset_index(drop=True)

## getting a glimpse of the data

In [5]:
df.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0.0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1.0
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0.0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0.0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1.0


## more information on the data
## the data shows number of rows and columns
## there is presence of missing values

In [6]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13281 entries, 0 to 13280
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      13281 non-null  object 
 1   Policy Start Date       13281 non-null  object 
 2   Policy End Date         13281 non-null  object 
 3   Gender                  12881 non-null  object 
 4   Age                     13281 non-null  int64  
 5   First Transaction Date  13281 non-null  object 
 6   No_Pol                  13281 non-null  int64  
 7   Car_Category            9171 non-null   object 
 8   Subject_Car_Colour      5622 non-null   object 
 9   Subject_Car_Make        10557 non-null  object 
 10  LGA_Name                6149 non-null   object 
 11  State                   6137 non-null   object 
 12  ProductName             13281 non-null  object 
 13  target                  12079 non-null  float64
dtypes: float64(1), int64(2), object(11)
me

## definations of the columns in the data

In [7]:
pd.read_csv('VariableDefinitions.csv')

,ID,Unique ID for the customer
0,Policy Start Date,Date policy started
1,Policy End Date,Date policy ended
2,Gender,Gender of the customer
3,Age,Age of the customer
4,ProductName,Name of Insurance policy
5,First Transaction Date,First date payment was made
6,No_Pol,Number of policies the customer has
7,Car_Category,Type of car
8,Subject_Car_Colour,Car colour
9,Subject_Car_Make,Car make


## number of null values

In [8]:
df.isnull().sum()

ID                           0
Policy Start Date            0
Policy End Date              0
Gender                     400
Age                          0
First Transaction Date       0
No_Pol                       0
Car_Category              4110
Subject_Car_Colour        7659
Subject_Car_Make          2724
LGA_Name                  7132
State                     7144
ProductName                  0
target                    1202
dtype: int64

## percentage of null values 

In [9]:
100*(df.isnull().sum()/len(df))

ID                         0.000000
Policy Start Date          0.000000
Policy End Date            0.000000
Gender                     3.011821
Age                        0.000000
First Transaction Date     0.000000
No_Pol                     0.000000
Car_Category              30.946465
Subject_Car_Colour        57.668850
Subject_Car_Make          20.510504
LGA_Name                  53.700776
State                     53.791130
ProductName                0.000000
target                     9.050523
dtype: float64

## renaming black in car make to avoid confusion

In [10]:
def car(x):
    if x=='Black':
        return 'Black_1'
    else:
        return x

In [11]:
df['Subject_Car_Make']=df['Subject_Car_Make'].apply(car)

In [12]:
df[df['Subject_Car_Make']=='Black_1']

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
6015,ID_I1S08AV,2010-01-05,2011-01-04,Male,36,2010-01-05,1,Saloon,NaN,Black_1,Somolu,Lagos,Car Classic,1.0
9732,ID_T4RS1MN,2010-01-05,2011-01-04,Male,36,2010-01-05,1,Saloon,NaN,Black_1,Somolu,Lagos,Car Classic,0.0


## cleaning the car category column

In [13]:
df['Car_Category'].isnull().sum()

4110

In [14]:
df[df['Gender'].isnull()]['Car_Category'].value_counts()

JEEP          72
Saloon        68
Motorcycle     6
Bus            6
Truck          6
Name: Car_Category, dtype: int64

## most null values in gender had a jeep

In [15]:
def category(dataframe):
    Gender=dataframe['Gender']
    Car_Category=dataframe['Car_Category']
    for i in dataframe.index:
        if pd.isnull(Gender[i]):
            Car_Category[i] = 'JEEP'
        else:
            Car_Category[i] = Car_Category[i]

In [16]:
df=df[['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target']]
category(df)

In [17]:
df['Car_Category'].isnull().sum()

3868

In [18]:
df[['Car_Category','Subject_Car_Colour','Subject_Car_Make']].value_counts().head(5)

Car_Category  Subject_Car_Colour  Subject_Car_Make
Saloon        Black               TOYOTA              572
              Silver              TOYOTA              320
JEEP          Black               TOYOTA              315
Saloon        Grey                TOYOTA              283
              Blue                TOYOTA              204
dtype: int64

In [19]:
def category(dataframe):
    Car_Category=dataframe['Car_Category']
    Subject_Car_Colour=dataframe['Subject_Car_Colour']
    Subject_Car_Make=dataframe['Subject_Car_Make']
    for i in dataframe.index:
        if pd.isnull(Car_Category[i]) and Subject_Car_Colour[i] == 'Black' and Subject_Car_Make[i] == 'TOYOTA':
            Car_Category[i] = 'Saloon'
        elif pd.isnull(Car_Category[i]) and Subject_Car_Colour[i] == 'Silver' and Subject_Car_Make[i] == 'TOYOTA':
            Car_Category[i] = 'Saloon'
        elif pd.isnull(Car_Category[i]) and Subject_Car_Colour[i] == 'Grey' and Subject_Car_Make[i] == 'TOYOTA':
            Car_Category[i] = 'Saloon'
        elif pd.isnull(Car_Category[i]) and Subject_Car_Colour[i] == 'Blue' and Subject_Car_Make[i] == 'TOYOTA':
            Car_Category[i] = 'Saloon'
            
        else:
            Car_Category[i] = Car_Category[i]

In [20]:
df=df[['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target']]
category(df)

In [21]:
df['Car_Category'].isnull().sum()

3764

In [22]:
df[['ProductName','LGA_Name', 'State','Car_Category']].value_counts().head()

ProductName       LGA_Name         State  Car_Category
Car Classic       Victoria Island  Lagos  JEEP            238
Customized Motor  Victoria Island  Lagos  JEEP            168
Car Classic       Ikeja            Lagos  Saloon          160
                  Victoria Island  Lagos  Saloon          155
                  Surulere         Lagos  Saloon          116
dtype: int64

In [23]:
def category(dataframe):
    Car_Category=dataframe['Car_Category']
    LGA=dataframe['LGA_Name']
    State=dataframe['State']
    for i in dataframe.index:
        if pd.isnull(Car_Category[i]) and LGA[i] == 'Victoria Island' and State[i] == 'Lagos':
            Car_Category[i] = 'JEEP' 
        elif pd.isnull(Car_Category[i]) and LGA[i] == 'Ikeja' and State[i] == 'Lagos':
            Car_Category[i] = 'Saloon' 
        else:
            Car_Category[i] = Car_Category[i]

In [24]:
df=df[['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target']]
category(df)

In [25]:
df['Car_Category'].isnull().sum()

3227

In [26]:
df[['Car_Category','Subject_Car_Make','ProductName']].value_counts().head(5)

Car_Category  Subject_Car_Make  ProductName
Saloon        TOYOTA            Car Classic    2386
                                CarSafe        1364
JEEP          TOYOTA            Car Classic     780
Saloon        Honda             CarSafe         407
                                Car Classic     334
dtype: int64

In [27]:
def category(dataframe):
    Car_Category=dataframe['Car_Category']
    Subject_Car=dataframe['Subject_Car_Make']
    Product=dataframe['ProductName']
    for i in dataframe.index:
        if pd.isnull(Car_Category[i]) and Subject_Car[i]=='TOYOTA' and Product[i]=='Car Classic':
            Car_Category[i] = 'Saloon' 
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='Lexus' and Product[i]=='Car Classic':
            Car_Category[i] = 'JEEP' 
        else:
            Car_Category[i] = Car_Category[i]

In [28]:
df=df[['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target']]
category(df)

In [29]:
df['Car_Category'].isnull().sum()

2995

In [30]:
df[['Car_Category','ProductName']].value_counts().head(5)

Car_Category  ProductName     
Saloon        Car Classic         3695
              CarSafe             2742
JEEP          Car Classic         2040
              Customized Motor     371
Saloon        Car Plus             276
dtype: int64

In [31]:
def category(dataframe):
    Car_Category=dataframe['Car_Category']
    Product=dataframe['ProductName']
    for i in dataframe.index:
        if pd.isnull(Car_Category[i]) and (Product[i]=='Car Classic' or Product[i]=='CarSafe' or Product[i]=='Car Plus') :
            Car_Category[i] = 'Saloon' 
        elif pd.isnull(Car_Category[i]) and Product[i]=='Customized Motor':
            Car_Category[i] = 'JEEP' 
        else:
            Car_Category[i] = Car_Category[i]

In [32]:
df=df[['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target']]
category(df)

In [33]:
df['Car_Category'].isnull().sum()

259

In [34]:
def category(dataframe):
    Car_Category=dataframe['Car_Category']
    Subject_Car=dataframe['Subject_Car_Make']
    for i in dataframe.index:
        if pd.isnull(Car_Category[i]) and Subject_Car[i]=='Iveco':
            Car_Category[i] = 'Truck' 
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='Honda':
            Car_Category[i] = 'Saloon'
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='DAF':
            Car_Category[i] = 'Truck'
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='TOYOTA':
            Car_Category[i] = 'Saloon'
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='Mitsubishi':
            Car_Category[i] = 'Pickup'
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='Scania':
            Car_Category[i] = 'JEEP'
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='Hyundai':
            Car_Category[i] = 'Bus'
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='Ford':
            Car_Category[i] = 'Pickup'
        elif pd.isnull(Car_Category[i]) and Subject_Car[i]=='Isuzu':
            Car_Category[i] = 'Pickup'
        else:
            Car_Category[i] = Car_Category[i]

In [35]:
df=df[['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target']]
category(df)

In [36]:
df['Car_Category'].isnull().sum()

148

In [37]:
df['Car_Category']=df['Car_Category'].fillna('Saloon')

In [38]:
df['Car_Category'].isnull().sum()

0

## filling missing gender values with the mode

In [39]:
df['Gender'].unique()

array(['Male', 'Female', 'Entity', 'Joint Gender', nan, 'NO GENDER',
       'NOT STATED', 'SEX'], dtype=object)

In [40]:
df['Gender'].value_counts().head(5)

Male            8356
Female          3679
Entity           300
Joint Gender     238
NOT STATED       190
Name: Gender, dtype: int64

In [41]:
df['Gender']=df['Gender'].fillna('Male')

In [42]:
mapper = {'Entity':'Other', 'Joint Gender':'Other', 'NOT STATED':'Other', 'NO GENDER': 'Other', 'SEX':"Other"}
df['Gender'] = df['Gender'].replace(mapper)

# Confirm mappings
df['Gender'].value_counts()

Male      8756
Female    3679
Other      846
Name: Gender, dtype: int64

In [43]:
df['Gender'].isna().sum()

0

## age column

In [44]:
df['Age'].unique()

array([   30,    79,    43,     2,    20,    37,    40,    71,    58,
          45,    51,   120,    38,    81,    69,    31,    70,    34,
           1,    41,    47,    36,    46,    48,    39,    63,    42,
          27,    64,    50,    53,    33,    23,    32,    59,    60,
          52,    54,    44,    11,    78,    28,    29,    57,    12,
          65,    35,    73,    56,    55,    26,    49,    18,    14,
          62,    25,    61,    72,    82,    67,     7,    10,    68,
          21,    76,    24,    66,     3,    93,    22,    77,    19,
          80,     9,     6,   -76,    -2,    89,    74,    75,    84,
          16,    90,   140,     8,     5,    13,    86,    15,   320,
         -12,    83,    85, -6099, -5939,     4,   -22,   144,   112,
          17,     0,    88,   -27,    87,   133,   102,   -51,   128,
         -26,   100], dtype=int64)

## dealing with outliers in age

In [45]:
Q1=np.percentile(df['Age'],25)
Q2=np.percentile(df['Age'],50)
Q3=np.percentile(df['Age'],75)
#formula
IQR=Q3-Q1
low_lim=Q1-1.5*IQR
upper_lim=Q3+1.5*IQR
print(f'Values not within the age of {low_lim,upper_lim} are outliers')

Values not within the age of (12.5, 72.5) are outliers


## replacing outliers with the median

In [46]:
def age(x):
    if x<=12.5 or x>=72.5:
        return Q2
    else:
        return x

In [47]:
df['Age']=df['Age'].apply(age)

In [48]:
df['Age'].unique()

array([30., 41., 43., 20., 37., 40., 71., 58., 45., 51., 38., 69., 31.,
       70., 34., 47., 36., 46., 48., 39., 63., 42., 27., 64., 50., 53.,
       33., 23., 32., 59., 60., 52., 54., 44., 28., 29., 57., 65., 35.,
       56., 55., 26., 49., 18., 14., 62., 25., 61., 72., 67., 68., 21.,
       24., 66., 22., 19., 16., 13., 15., 17.])

## cleaning car colour

In [49]:
df[['Car_Category','Subject_Car_Colour','Subject_Car_Make','LGA_Name']].value_counts().head(20)

Car_Category  Subject_Car_Colour  Subject_Car_Make  LGA_Name       
JEEP          Black               TOYOTA            Victoria Island    51
Saloon        Black               TOYOTA            Victoria Island    48
JEEP          As Attached         TOYOTA            Victoria Island    48
Saloon        Black               TOYOTA            Ikeja              40
JEEP          Black               Lexus             Victoria Island    27
Saloon        Black               TOYOTA            Lagos Mainland     25
                                                    Abuja Municipal    24
JEEP          Black               TOYOTA            Eti-Osa            24
                                  Mercedes          Victoria Island    23
Saloon        Grey                TOYOTA            Victoria Island    22
              Black               TOYOTA            Surulere           21
JEEP          Black               Hyundai           Victoria Island    20
                                  Honda     

In [50]:
def category(dataframe):
    Car_Category=dataframe['Car_Category']
    Subject_Car_Colour=dataframe['Subject_Car_Colour']
    Subject_Car_Make=dataframe['Subject_Car_Make']
    LGA=dataframe['LGA_Name']
    for i in dataframe.index:
        if pd.isnull(Subject_Car_Colour[i]) and Car_Category[i]=='JEEP' and Subject_Car_Make[i] == 'TOYOTA' and LGA[i]=='Victoria Island':
            Subject_Car_Colour[i] = 'Black'
        elif pd.isnull(Subject_Car_Colour[i]) and Car_Category[i]=='JEEP' and Subject_Car_Make[i]=='TOYOTA' and LGA[i]=='Victoria Island':
            Subject_Car_Colour[i] = 'As Attached' 
        elif pd.isnull(Subject_Car_Colour[i]) and Car_Category[i]=='Saloon' and Subject_Car_Make[i]=='TOYOTA' and LGA[i]=='Ikeja':
            Subject_Car_Colour[i] = 'Black'
        elif pd.isnull(Subject_Car_Colour[i]) and Car_Category[i]=='JEEP' and Subject_Car_Make[i]=='Lexus' and LGA[i]=='Victoria Island':
            Subject_Car_Colour[i] = 'Black'
        elif pd.isnull(Subject_Car_Colour[i]) and Car_Category[i]=='Saloon' and Subject_Car_Make[i]=='TOYOTA' and LGA[i]=='Lagos Mainland':
            Subject_Car_Colour[i] = 'Grey'
        elif pd.isnull(Subject_Car_Colour[i]) and Car_Category[i]=='Saloon' and Subject_Car_Make[i]=='TOYOTA' and LGA[i]=='Ikeja':
            Subject_Car_Colour[i] = 'Silver'
        elif pd.isnull(Subject_Car_Colour[i]) and Car_Category[i]=='Saloon' and Subject_Car_Make[i]=='TOYOTA' and LGA[i]=='Ikeja':
            Subject_Car_Colour[i] = 'Grey'
            
        else:
            Subject_Car_Colour[i]=Subject_Car_Colour[i]

In [51]:
df=df[['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target']]
category(df)

In [52]:
df['Subject_Car_Colour'].isnull().sum()

7493

In [53]:
def category(dataframe):
    Subject_Car_Colour=dataframe['Subject_Car_Colour']
    Product=dataframe['ProductName']
    for i in dataframe.index:
        if pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='Car Classic':
            Subject_Car_Colour[i] = 'Black'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='Customized Motor':
            Subject_Car_Colour[i] = 'Black'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='Car Plus':
            Subject_Car_Colour[i] = 'Black'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='CVTP':
            Subject_Car_Colour[i] = 'White'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='CVTP':
            Subject_Car_Colour[i] = 'CarSafe'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='Muuve':
            Subject_Car_Colour[i] = 'As Attached'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='CarFlex':
            Subject_Car_Colour[i] = 'As Attached'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='Motor Cycle':
            Subject_Car_Colour[i] = 'Black'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='Car Vintage':
            Subject_Car_Colour[i] = 'Black'
        elif pd.isnull(Subject_Car_Colour[i])  and  Product[i]=='CarSafe':
            Subject_Car_Colour[i] = 'Black'
        else:
            Subject_Car_Colour[i] = Subject_Car_Colour[i]

In [54]:
df=df[['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target']]
category(df)

In [55]:
df['Subject_Car_Colour'].isnull().sum()

0

## filling null Subject CarMake with the mode

In [56]:
df['Subject_Car_Make'].value_counts().head(5)

TOYOTA      5466
Honda       1146
Lexus        654
Mercedes     577
Hyundai      512
Name: Subject_Car_Make, dtype: int64

In [57]:
df['Subject_Car_Make']=df['Subject_Car_Make'].fillna('TOYOTA')

## filling LGA name with the mode

In [58]:
df['LGA_Name'].value_counts().head()

Victoria Island    1284
Ikeja               438
Surulere            306
Abuja Municipal     257
Lagos Mainland      245
Name: LGA_Name, dtype: int64

In [59]:
df['LGA_Name']=df['LGA_Name'].fillna('Victoria Island')

## filling null state column with the mode

In [60]:
#state column
df['State']=df['State'].fillna('Lagos')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13281 entries, 0 to 13280
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      13281 non-null  object 
 1   Policy Start Date       13281 non-null  object 
 2   Policy End Date         13281 non-null  object 
 3   Gender                  13281 non-null  object 
 4   Age                     13281 non-null  float64
 5   First Transaction Date  13281 non-null  object 
 6   No_Pol                  13281 non-null  int64  
 7   Car_Category            13281 non-null  object 
 8   Subject_Car_Colour      13281 non-null  object 
 9   Subject_Car_Make        13281 non-null  object 
 10  LGA_Name                13281 non-null  object 
 11  State                   13281 non-null  object 
 12  ProductName             13281 non-null  object 
 13  target                  12079 non-null  float64
dtypes: float64(2), int64(1), object(11)
me

## Change each column to its appriopriate datatype

In [62]:

date_cols = [col for col in df.columns if 'Date' in col]
num_cols = ['Age', 'No_Pol']
cat_cols = [col for col in df.columns if col not in date_cols+num_cols+['ID', 'target']]

for col in df.columns:
  if col in date_cols:
    df[col] = pd.to_datetime(df[col])
  elif col in cat_cols:
    df[col] = df[col].astype('category')

# Confirm whether the changes have been applied successfully
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13281 entries, 0 to 13280
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID                      13281 non-null  object        
 1   Policy Start Date       13281 non-null  datetime64[ns]
 2   Policy End Date         13281 non-null  datetime64[ns]
 3   Gender                  13281 non-null  category      
 4   Age                     13281 non-null  float64       
 5   First Transaction Date  13281 non-null  datetime64[ns]
 6   No_Pol                  13281 non-null  int64         
 7   Car_Category            13281 non-null  category      
 8   Subject_Car_Colour      13281 non-null  category      
 9   Subject_Car_Make        13281 non-null  category      
 10  LGA_Name                13281 non-null  category      
 11  State                   13281 non-null  category      
 12  ProductName             13281 non-null  catego

## feature engineering date time columns

In [63]:
df['Policy_Start']=df['Policy Start Date'].dt.year
df['Policy_Start_Month']=df['Policy Start Date'].dt.month
df['Policy_Start_Day']=df['Policy Start Date'].dt.day

In [64]:
df['Policy_End_Year']=df['Policy End Date'].dt.year
df['Policy_End_Month']=df['Policy End Date'].dt.month
df['Policy_End_Day']=df['Policy End Date'].dt.day


In [65]:
df.drop(['Policy Start Date','Policy End Date','First Transaction Date',],axis=1,inplace=True)

## developing the machine learning algorithm

## converting categorical variables to numeric

In [66]:
df=pd.get_dummies(df.drop('ID',axis=1),drop_first=True)

## Separate train and test data from the combined dataframe

In [67]:
train_df = df[:ntrain]
test_df = df[ntrain:]

## Check the shapes of the split dataset

In [68]:
train_df.shape, test_df.shape

((12079, 535), (1202, 535))

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
X=train_df.drop(['target'],axis=1)
y=train_df['target'].astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [71]:
from sklearn.ensemble import RandomForestClassifier

In [72]:
rfc = RandomForestClassifier(n_estimators=200)

In [73]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200)

In [74]:
rfc.pred = rfc.predict(X_test)

In [75]:
from sklearn.metrics import confusion_matrix,classification_report

In [76]:
print(confusion_matrix(y_test,rfc.pred))
print('--'*50)
print(classification_report(y_test,rfc.pred))

[[3045  145]
 [ 359   75]]
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      3190
           1       0.34      0.17      0.23       434

    accuracy                           0.86      3624
   macro avg       0.62      0.56      0.58      3624
weighted avg       0.83      0.86      0.84      3624



## Balancing the target variables

In [77]:
from sklearn.utils import resample

## Separate majority and minority classes

In [78]:
train_majority = train_df[train_df['target']==0.0]
train_minority = train_df[train_df['target']==1.0]

## Upsample minority class
## sample with replacement to match majority class

In [79]:
train_minority_upsampled = resample(train_minority, replace=True,n_samples=10624,random_state=123)

## Combine majority class with upsampled minority class

In [80]:
train_upsampled = pd.concat([train_majority,train_minority_upsampled])

## performing the algorithm again

In [81]:
rfc = RandomForestClassifier(n_estimators=200)

In [82]:
X=train_upsampled.drop(['target'],axis=1)
y=train_upsampled['target'].astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [83]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200)

In [84]:
rfc.pred = rfc.predict(X_test)

In [85]:
print(confusion_matrix(y_test,rfc.pred))
print('--'*50)
print(classification_report(y_test,rfc.pred))

[[2841  348]
 [  36 3150]]
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.89      0.94      3189
           1       0.90      0.99      0.94      3186

    accuracy                           0.94      6375
   macro avg       0.94      0.94      0.94      6375
weighted avg       0.94      0.94      0.94      6375



## model on test data

In [86]:
X_test=test_df.drop(['target'],axis=1)
target=rfc.predict(X_test)
test['target']=target
ID=test['ID']
test1=test[['ID','target']]


In [87]:
test['target'].value_counts()

0    1108
1      94
Name: target, dtype: int64

## using decision trees

In [88]:
X=train_upsampled.drop(['target'],axis=1)
y=train_upsampled['target'].astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [89]:
from sklearn.tree import DecisionTreeClassifier
dtree= DecisionTreeClassifier()
dtree.fit(X_train,y_train)
predictions = dtree.predict(X_test)
print(confusion_matrix(y_test,predictions))
print('--'*50)
print(classification_report(y_test,predictions))

[[2638  545]
 [  16 3176]]
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.83      0.90      3183
           1       0.85      0.99      0.92      3192

    accuracy                           0.91      6375
   macro avg       0.92      0.91      0.91      6375
weighted avg       0.92      0.91      0.91      6375



## model on test data

In [90]:
X_test=test_df.drop(['target'],axis=1)
target=dtree.predict(X_test)
test['target']=target
ID=test['ID']
test1=test[['ID','target']]
test['target'].value_counts()

0    1022
1     180
Name: target, dtype: int64